## Unity to Mujoco Creature Converter

A script to convert JSON from the UTMIST's Virtual Creatures Unity project into Mujoco, which uses a nested XML file format.

In [1022]:
from dataclasses import dataclass
from __future__ import annotations
from jinja2 import Environment, FileSystemLoader
import json
import os
from typing import List

In [1023]:
# Import creature json
body_parts = {}

filename = 'creature_configs/two_legged_shlawg_blueprint.json'
with open(filename) as f:
    data = json.load(f)
    for entry in data:
        id = entry["UniqueId"]
        body_parts[id] = entry    

### Breaking Down the Creature

We see that each body part is composed of the following fields:

|Field|Type|Notes|
|--|--|--|
| UniqueId | Int | A unique ID for the body part |
| TypeId | Int | ??? |
| ParentUniqueId | Int \| None | |
| Position | Vector3 | |
| LocalPosition | Vector3 | Position relative to parent |
| Rotation | Vector3 | |
| LocalRotation | Vector3 | Rotation relative to parent |
| Size | Vector3 | Size of part |
| JointType | 'hinge' \| 'fixed' \| None |  |
| JointAnchorPos | Vector3 \| None | Position of the joint (relative to child, presumeably) |
| JointAxis | Vector3 \| None | 0 or 1 for whether joint is free for each axis |
| Color | Vector3 | Hex values |

**Notes**:
- All vectors in the JSON file store their magnitudes and sometimes normalization, this seems cached and not actually useful (sometimes the normalizations have normalizations?)
- Rotation is presumeably Euler rotations

**Questions**:
- Check what `TypeID` does
- For whatever reason, `LocalPosition` is the same as `Position` even when parent position is not (0,0,0)

In [1024]:
# Define an interface to interact with
@dataclass
class Vector3():
    x: float
    y: float
    z: float
    
    def __str__(self):
        return f"{self.x} {self.y} {self.z}"
    
    @staticmethod
    def from_json(json_dict):
        return Vector3(json_dict["x"], json_dict["y"], json_dict["z"])

@dataclass
class Joint():
    type: str
    anchor_position: Vector3
    axis: Vector3 | None

### Coordinate Conversions

#### Position

For most axes, we use

`2 * (unity_pos_child - unity_pos_immediate_parent)`

However, for the vertical axis, we have:

```
2 * (unity_pos_child - unity_pos_immediate_parent) + size_child - size_parent
```

We are doing this because in Unity, gameobjects are centered on the center of the bottom face, but in Mujoco, the bodies are centered in the very center of the object

#### 

In [1025]:
@dataclass
class BodyPart():
    id: int
    parent: BodyPart | None
    children: List[BodyPart]
    position: Vector3
    rotation: Vector3
    size: Vector3
    color: Vector3
    joint: Joint | None
    
    ################## Coordinate Conversions ##################
    
    def mujoco_position(self):
        parent_position = self.parent.position if self.parent else Vector3(0, 0, 0)
        parent_size = self.parent.size if self.parent else Vector3(0, 0, 0)
        
        mujoco_x = 2 * (self.position.x - parent_position.x)
        mujoco_y = 2 * (self.position.z - parent_position.z)
        mujoco_z = 2 * (self.position.y - parent_position.y) + self.size.y - parent_size.y
        
        return Vector3(mujoco_x, mujoco_y, mujoco_z)
    
    def mujoco_size(self):
        return Vector3(self.size.x, self.size.z, self.size.y)
    
    def mujoco_joint_position(self):
        if (self.joint):
            return Vector3(-self.joint.anchor_position.x, -self.joint.anchor_position.z, -self.joint.anchor_position.y)
        
    def mujoco_axis(self):
        if (self.joint):
            x_axis = 1 if self.joint.axis.x else 0
            y_axis = 1 if self.joint.axis.z else 0
            z_axis = 1 if self.joint.axis.y else 0
            return Vector3(x_axis, y_axis, z_axis)
        

In [1026]:
def body_part_from_json(json_dict, parent=None):
        if json_dict["JointType"] == "hinge":
            joint = Joint(
                type=json_dict["JointType"],
                anchor_position=Vector3.from_json(json_dict["JointAnchorPos"]),
                axis=Vector3.from_json(json_dict["JointAxis"])
            )
        elif json_dict["JointType"] == "fixed":
            joint = Joint(
                type=json_dict["JointType"],
                anchor_position=Vector3.from_json(json_dict["JointAnchorPos"]),
                axis=None
            )
        else:
            joint = None
        
        return BodyPart(
            id=json_dict["UniqueId"],
            parent=parent,
            children=[],
            position=Vector3.from_json(json_dict["Position"]),
            rotation=Vector3.from_json(json_dict["Rotation"]),
            size=Vector3.from_json(json_dict["Size"]),
            color=Vector3.from_json(json_dict["Color"]),
            joint=joint
        )

root = body_part_from_json(body_parts[0])
body_parts_list = []

def recursively_assemble_creature(root: BodyPart, body_parts_dict):
    for body_part in body_parts_dict.values():
        if body_part["ParentUniqueId"] == root.id:
            root.children.append(body_part_from_json(body_part, root))
            recursively_assemble_creature(root.children[-1], body_parts_dict)
    body_parts_list.append(root)
            
recursively_assemble_creature(root, body_parts)
body_parts_list = sorted(body_parts_list, key=lambda bp: bp.id)

In [1027]:
def print_body_part(bp: BodyPart, layer=0):
    def layered_print(str):
        prefix = '    ' * layer
        print(f"{prefix}{str}")
    
    layered_print(f"\033[1m\033[34mBody Part {bp.id}\033[0m")
    layered_print(f"ID:         {bp.id}")
    layered_print(f"Position:   {bp.mujoco_position()}")
    layered_print(f"Rotation:   {bp.rotation}")
    layered_print(f"Size:       {bp.mujoco_size()}")
    
    if bp.joint:
        layered_print(f"Joint:      Joint Type: {bp.joint.type}")
        layered_print(f"            Joint Anchor: {bp.mujoco_joint_position()}")
        layered_print(f"            Joint Axis: {bp.joint.axis}")
    else:
        layered_print(f"Joint:      None")
    
    for child in bp.children:
        print_body_part(child, layer + 1)

print_body_part(root)

Body Part 0
ID:         0
Position:   0.0 0.0 1.47805929
Rotation:   0.000158063776 0.000541241141 0.00360564
Size:       0.7242545 0.681036532 1.47805929
Joint:      None
    Body Part 1
    ID:         1
    Position:   0.0 0.0 -1.15261459
    Rotation:   0.000158063776 -0.0005412411 -0.00360564
    Size:       0.5204368 0.4150174 0.3254447
    Joint:      Joint Type: hinge
                Joint Anchor: 0.4970588 0.500000536 -0.4954357
                Joint Axis: 0.0 0.0 -1.0


In [1028]:
template_dir = os.path.join('./creature_template')
loader = FileSystemLoader(template_dir)
jinja_env = Environment(loader=loader)

# Forward declaration
jinja_env.filters.update(render_body_part=lambda x: "")

In [1029]:
with open("./creature_template/body_template.xml", "r") as file:
    body_template = jinja_env.from_string(file.read())

def render_body_part(body_part: BodyPart):
    rendered_template = body_template.render({
        "bp": body_part
    })

    return rendered_template

jinja_env.filters.update(render_body_part=render_body_part)

print(render_body_part(root))

<body name="seg0" pos="0.0 0.0 1.47805929">
    <geom name="seg0_geom" type="box" pos="0 0 0" size="0.7242545 0.681036532 1.47805929" euler="0.000158063776 0.000541241141 0.00360564" />
    <site name="seg0_site" type="box" pos="0 0 0" size="0.09 0.09 0.24" zaxis="0.2 0.2 0" rgba="1 1 0 0" />

    <body name="seg1" pos="0.0 0.0 -1.15261459">
        <joint name="seg0_to_1" range="-75 75" type="hinge" axis="0 1 0" pos="0.4970588 0.500000536 -0.4954357 "/>
        <geom name="seg1_geom" type="box" pos="0 0 0" size="0.5204368 0.4150174 0.3254447" euler="0.000158063776 -0.0005412411 -0.00360564" />
        <site name="seg1_site" type="box" pos="0 0 0" size="0.09 0.09 0.24" zaxis="0.2 0.2 0" rgba="1 1 0 0" />
    </body>
</body>


In [1030]:
with open("./creature_template/template.xml", "r") as file:
    template = jinja_env.from_string(file.read())

rendered_template = template.render({
    "model_name": "Fish",
    "bp": root,
    "bp_list": body_parts_list
})

print(rendered_template)

<mujoco model="Fish">
    <compiler angle="degree" />
    <default>
		<motor ctrlrange="-1.0 1.0" ctrllimited="true" gear="1500" />
        <geom friction="1 0.5 0.5" solref=".02 1" solimp="0 .8 .01" material="self" density="50.0" />
        <joint limited="true" armature="1" damping="1" stiffness="1" solreflimit=".04 1"
            solimplimit="0 .8 .03" />
        <default class="rangefinder">
            <site type="capsule" size=".05 .5" rgba="1 0 0 .4" group="4" />
        </default>
    </default>
    <asset>
        <material name="self" rgba=".8 .6 .4 1" />
    </asset>

    <worldbody>
        <body name="seg0" pos="0.0 0.0 1.47805929">
            
            <camera name="floating" pos="-2 0 1" xyaxes="0 -1 0 .5 0 1" mode="trackcom" fovy="90" />

            <!-- KEEP THIS -->
            <camera name="egocentric" pos=".25 0 .11" xyaxes="0 -1 0 0 0 1" fovy="90" />

            <!-- Sensors as we need -->
            <site name="rf_xp" class="rangefinder" pos="0.25 0 0.11" z

In [1031]:
with open('creature_configs/two_legged_shlawg_blueprint.xml', 'w') as file:
    file.write(rendered_template)